In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index import SimpleDirectoryReader, Document, GPTListIndex

documents = SimpleDirectoryReader('./meeting_notes').load_data()

index = GPTListIndex.from_documents(documents)

query_engine = index.as_query_engine(
    response_mode="tree_summarize"
)

summarization_query = "This text is a transcript from a virtual video meeting. Summarize the meeting, including the main points, decisions made, and action items."
response = query_engine.query(summarization_query)

print(response)